In [1]:
import io
import numpy as np
import PIL
from PIL import Image
import requests
import torch
import openpifpaf
from os import listdir
from os.path import isfile, join
from tensorflow import keras
import os

In [6]:
fpath = "data/"
allfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
print(len(allfiles), "pictures found.")

1286 pictures found.


In [5]:
i = 0
os.makedirs("processed/", exist_ok=True)
for f in allfiles:
    pil_im = Image.open(fpath + f).convert("RGB")
    im = np.asarray(pil_im)
    predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')
    predictions, gt_anns, image_meta = predictor.pil_image(pil_im)
    if len(predictions) == 0: continue
    
    for prediction in predictions:
    
        if prediction.score < 0.65: continue
        i += 1
        bbox = prediction.bbox()

        padding = 0.1
        l_x = bbox[0] - padding * bbox[2]
        l_y = bbox[1] - padding * bbox[3]
        r_x = l_x + (1 + 2 * padding) * bbox[2]
        r_y = l_y + (1 + 2 * padding) * bbox[3]

        if l_x < 0: l_x = 0
        if l_y < 0: l_y = 0
        if r_x > pil_im.width: r_x = pil_im.width
        if r_y > pil_im.height: r_y = pil_im.height

        cropped = pil_im.crop((l_x, l_y, r_x, r_y))
        cropped = cropped.resize((100, 200))

        cropped.save(f"processed/procd_{str(i)}.png")
        print("Image with {} confidence saved".format(prediction.score))
        i += 1

Image with 0.8865676820278167 confidence saved
Image with 0.6514355799426204 confidence saved
Image with 0.8988518533499344 confidence saved
Image with 0.9171180647352467 confidence saved
Image with 0.9103780362917029 confidence saved
Image with 0.8650361636410588 confidence saved
Image with 0.8321013295132181 confidence saved
Image with 0.7827035357122835 confidence saved
Image with 0.8887803321299346 confidence saved
Image with 0.8586710691452026 confidence saved
Image with 0.9061246047849241 confidence saved
Image with 0.8737391181614087 confidence saved
Image with 0.7723681887854701 confidence saved
Image with 0.7211533784866333 confidence saved
Image with 0.7717550008193306 confidence saved
Image with 0.697375546330991 confidence saved
Image with 0.7422876746758171 confidence saved
Image with 0.6758464030597521 confidence saved
Image with 0.7451401624990546 confidence saved
Image with 0.7349775070729463 confidence saved
Image with 0.6957149635190549 confidence saved
Image with 0.7

In [7]:
def predict_smato(cropped_image, model):
    '''Returns True (has smartphone) or False based on the model
    '''
    im = np.asarray(cropped_image) * 1.0/255.0
    im = im.reshape((1, 224, 224, 3))
    prediction = model.predict(im)[0][0]
    print("Raw prediction results: ", prediction)
    return prediction >= 0.5

In [8]:
# load model
model_path = "saved_models/smato_efficientnet_m2/"
model = keras.models.load_model(model_path, compile = False)

In [9]:
fpath = "processed/"
allfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
print(len(allfiles), "processed pictures found.")

4845 processed pictures found.


In [11]:
# sort into two different folders basde on trained model
# this reduces time to manually sort them
# after this, manually fix the classes
i = 0
for f in allfiles:
    cropped_image = Image.open(fpath + f).convert("RGB").resize((224, 224))
    result = predict_smato(cropped_image, model)
    print("Holding smartphone: ", result)
    cropped_image = cropped_image.resize((100, 200))
    if result:
        cropped_image.save(f"with/1true_{i}.png")
    else:
        cropped_image.save(f"without/1false_{i}.png")
    i += 1

Raw prediction results:  0.0019416707
Holding smartphone:  False
Raw prediction results:  0.006952178
Holding smartphone:  False
Raw prediction results:  0.00017666696
Holding smartphone:  False
Raw prediction results:  0.003038728
Holding smartphone:  False
Raw prediction results:  0.05556421
Holding smartphone:  False
Raw prediction results:  0.120041065
Holding smartphone:  False
Raw prediction results:  0.024631022
Holding smartphone:  False
Raw prediction results:  1.5298193e-08
Holding smartphone:  False
Raw prediction results:  0.00012141285
Holding smartphone:  False
Raw prediction results:  0.0026775415
Holding smartphone:  False
Raw prediction results:  0.013518782
Holding smartphone:  False
Raw prediction results:  0.39732924
Holding smartphone:  False
Raw prediction results:  9.2571434e-05
Holding smartphone:  False
Raw prediction results:  0.017670037
Holding smartphone:  False
Raw prediction results:  0.0008439509
Holding smartphone:  False
Raw prediction results:  0.7117

Raw prediction results:  0.755952
Holding smartphone:  True
Raw prediction results:  0.009750252
Holding smartphone:  False
Raw prediction results:  0.75481015
Holding smartphone:  True
Raw prediction results:  0.37234133
Holding smartphone:  False
Raw prediction results:  0.0023762612
Holding smartphone:  False
Raw prediction results:  1.03814245e-05
Holding smartphone:  False
Raw prediction results:  0.020952852
Holding smartphone:  False
Raw prediction results:  1.4578361e-05
Holding smartphone:  False
Raw prediction results:  0.024671035
Holding smartphone:  False
Raw prediction results:  0.042322293
Holding smartphone:  False
Raw prediction results:  3.862632e-05
Holding smartphone:  False
Raw prediction results:  0.03826457
Holding smartphone:  False
Raw prediction results:  1.624914e-05
Holding smartphone:  False
Raw prediction results:  0.001499106
Holding smartphone:  False
Raw prediction results:  0.004242274
Holding smartphone:  False
Raw prediction results:  6.052855e-10
Ho

Raw prediction results:  0.0005542556
Holding smartphone:  False
Raw prediction results:  0.003355229
Holding smartphone:  False
Raw prediction results:  0.06220221
Holding smartphone:  False
Raw prediction results:  0.010917049
Holding smartphone:  False
Raw prediction results:  0.0018899381
Holding smartphone:  False
Raw prediction results:  0.003658979
Holding smartphone:  False
Raw prediction results:  0.0058874837
Holding smartphone:  False
Raw prediction results:  7.364104e-05
Holding smartphone:  False
Raw prediction results:  0.85465664
Holding smartphone:  True
Raw prediction results:  0.5316493
Holding smartphone:  True
Raw prediction results:  0.0005462986
Holding smartphone:  False
Raw prediction results:  0.0001109197
Holding smartphone:  False
Raw prediction results:  4.939679e-07
Holding smartphone:  False
Raw prediction results:  1.8216246e-05
Holding smartphone:  False
Raw prediction results:  0.0002025423
Holding smartphone:  False
Raw prediction results:  0.77348584


Raw prediction results:  0.0011643199
Holding smartphone:  False
Raw prediction results:  0.0005070321
Holding smartphone:  False
Raw prediction results:  0.009073154
Holding smartphone:  False
Raw prediction results:  0.03920138
Holding smartphone:  False
Raw prediction results:  0.4524917
Holding smartphone:  False
Raw prediction results:  0.004194515
Holding smartphone:  False
Raw prediction results:  0.26326078
Holding smartphone:  False
Raw prediction results:  0.0004986568
Holding smartphone:  False
Raw prediction results:  0.87695825
Holding smartphone:  True
Raw prediction results:  0.01913992
Holding smartphone:  False
Raw prediction results:  0.0025211496
Holding smartphone:  False
Raw prediction results:  0.114670634
Holding smartphone:  False
Raw prediction results:  0.13244058
Holding smartphone:  False
Raw prediction results:  0.0016267558
Holding smartphone:  False
Raw prediction results:  0.0020696505
Holding smartphone:  False
Raw prediction results:  3.6595924e-05
Hol

Raw prediction results:  0.0033860563
Holding smartphone:  False
Raw prediction results:  1.0881313e-06
Holding smartphone:  False
Raw prediction results:  0.005675196
Holding smartphone:  False
Raw prediction results:  0.04219935
Holding smartphone:  False
Raw prediction results:  0.0002518208
Holding smartphone:  False
Raw prediction results:  5.032907e-06
Holding smartphone:  False
Raw prediction results:  0.0005448961
Holding smartphone:  False
Raw prediction results:  3.035734e-06
Holding smartphone:  False
Raw prediction results:  0.0004974984
Holding smartphone:  False
Raw prediction results:  0.017515432
Holding smartphone:  False
Raw prediction results:  5.349655e-07
Holding smartphone:  False
Raw prediction results:  0.012541418
Holding smartphone:  False
Raw prediction results:  3.0288886e-05
Holding smartphone:  False
Raw prediction results:  0.78156507
Holding smartphone:  True
Raw prediction results:  0.23808596
Holding smartphone:  False
Raw prediction results:  0.029096

...

Raw prediction results:  1.04288205e-08
Holding smartphone:  False
Raw prediction results:  4.3758555e-06
Holding smartphone:  False
Raw prediction results:  2.6911366e-05
Holding smartphone:  False
Raw prediction results:  0.00012043596
Holding smartphone:  False
Raw prediction results:  0.08053471
Holding smartphone:  False
Raw prediction results:  0.011788415
Holding smartphone:  False
Raw prediction results:  0.015853463
Holding smartphone:  False
Raw prediction results:  3.046058e-06
Holding smartphone:  False
Raw prediction results:  0.010041129
Holding smartphone:  False
Raw prediction results:  0.029925292
Holding smartphone:  False
Raw prediction results:  0.04103812
Holding smartphone:  False
Raw prediction results:  0.013243549
Holding smartphone:  False
Raw prediction results:  0.827453
Holding smartphone:  True
Raw prediction results:  0.000670393
Holding smartphone:  False
Raw prediction results:  0.7662847
Holding smartphone:  True
Raw prediction results:  0.01006087
Hol

Raw prediction results:  0.86077315
Holding smartphone:  True
Raw prediction results:  0.011444926
Holding smartphone:  False
Raw prediction results:  0.89557046
Holding smartphone:  True
Raw prediction results:  0.004356736
Holding smartphone:  False
Raw prediction results:  0.04968084
Holding smartphone:  False
Raw prediction results:  0.0020916136
Holding smartphone:  False
Raw prediction results:  0.041017387
Holding smartphone:  False
Raw prediction results:  6.6447515e-06
Holding smartphone:  False
Raw prediction results:  0.121747494
Holding smartphone:  False
Raw prediction results:  0.0004897761
Holding smartphone:  False
Raw prediction results:  0.00012475302
Holding smartphone:  False
Raw prediction results:  0.12367894
Holding smartphone:  False
Raw prediction results:  0.01601875
Holding smartphone:  False
Raw prediction results:  0.17308134
Holding smartphone:  False
Raw prediction results:  0.51306486
Holding smartphone:  True
Raw prediction results:  0.015990527
Holding

Raw prediction results:  0.00948019
Holding smartphone:  False
Raw prediction results:  7.2705916e-07
Holding smartphone:  False
Raw prediction results:  9.325836e-05
Holding smartphone:  False
Raw prediction results:  0.007093939
Holding smartphone:  False
Raw prediction results:  0.011298802
Holding smartphone:  False
Raw prediction results:  0.41112462
Holding smartphone:  False
Raw prediction results:  2.3070157e-05
Holding smartphone:  False
Raw prediction results:  0.00020583736
Holding smartphone:  False
Raw prediction results:  0.02209213
Holding smartphone:  False
Raw prediction results:  0.0053246133
Holding smartphone:  False
Raw prediction results:  0.09133868
Holding smartphone:  False
Raw prediction results:  0.0013605859
Holding smartphone:  False
Raw prediction results:  0.0005902008
Holding smartphone:  False
Raw prediction results:  0.043390907
Holding smartphone:  False
Raw prediction results:  0.44151354
Holding smartphone:  False
Raw prediction results:  0.87767804

Raw prediction results:  0.0035007244
Holding smartphone:  False
Raw prediction results:  0.53801215
Holding smartphone:  True
Raw prediction results:  0.032724008
Holding smartphone:  False
Raw prediction results:  9.679777e-07
Holding smartphone:  False
Raw prediction results:  2.2561748e-05
Holding smartphone:  False
Raw prediction results:  0.021523444
Holding smartphone:  False
Raw prediction results:  0.0021885722
Holding smartphone:  False
Raw prediction results:  0.0049827923
Holding smartphone:  False
Raw prediction results:  2.6374872e-07
Holding smartphone:  False
Raw prediction results:  0.007859159
Holding smartphone:  False
Raw prediction results:  2.8226486e-06
Holding smartphone:  False
Raw prediction results:  0.002863137
Holding smartphone:  False
Raw prediction results:  0.76590943
Holding smartphone:  True
Raw prediction results:  0.0031654658
Holding smartphone:  False
Raw prediction results:  2.5576574e-06
Holding smartphone:  False
Raw prediction results:  0.0005

Raw prediction results:  7.0598667e-06
Holding smartphone:  False
Raw prediction results:  0.00011156868
Holding smartphone:  False
Raw prediction results:  0.0041867406
Holding smartphone:  False
Raw prediction results:  0.7847586
Holding smartphone:  True
Raw prediction results:  0.80653024
Holding smartphone:  True
Raw prediction results:  0.0032327427
Holding smartphone:  False
Raw prediction results:  0.82157576
Holding smartphone:  True
Raw prediction results:  0.017500494
Holding smartphone:  False
Raw prediction results:  0.0019442617
Holding smartphone:  False
Raw prediction results:  0.96171963
Holding smartphone:  True
Raw prediction results:  0.0040708315
Holding smartphone:  False
Raw prediction results:  0.29971135
Holding smartphone:  False
Raw prediction results:  0.31029555
Holding smartphone:  False


In [12]:
# add flipped images to augment data
fpath = "with/"
allfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
print(len(allfiles), "with smartphone pictures found.")

for f in allfiles:
    original_img = Image.open(fpath + f)
    horz_img = original_img.transpose(method=Image.FLIP_LEFT_RIGHT)
    horz_img.save(fpath + "flipped_" + f)

565 with smartphone pictures found.


In [21]:
# add flipped images to augment data
fpath = "without/"
allfiles = [f for f in listdir(fpath) if isfile(join(fpath, f))]
print(len(allfiles), "without smartphone pictures found.")

for f in allfiles:
    original_img = Image.open(fpath + f)
    horz_img = original_img.transpose(method=Image.FLIP_LEFT_RIGHT)
    horz_img.save(fpath + "flipped_" + f)

125 without smartphone pictures found.
